<a href="https://colab.research.google.com/github/viercame/webscrapping/blob/main/Exito_webscrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium

In [ ]:
import sys
import csv
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import asyncio
from bs4 import BeautifulSoup
import json
import re

from google.colab import drive
drive.mount('drive')

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time

#options = Options()
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
driver = webdriver.Chrome(options=options)

paginas=[
         'despensa/arroz-granos-y-pastas',
         'despensa/aceites-y-vinagres',
         'despensa/azucar-panela-y-endulzante',
         'despensa/enlatados-y-conservas',
         'despensa/cereales-y-granolas',
         'despensa/cafe-chocolates-infusiones',
         'despensa/harinas-y-mezclas-para-preparar',
         'despensa/salsas-condimentos-y-sopas',
         'nunestrasmarcas',
         'lacteos-huevos-y-refrigerados',
         'vinos-y-licores',
         'aseo-del-hogar',
         'delicatessen',
         'congelados',
         'pollo-carne-y-pescado',
         'panaderia-y-reposteria',
         'snacks'
] # hay que ahondar en despensa y en lacteos huevos y refrigerados.

dict_param = [
    'productName',
    'brand',
    'link',
    'productId',
    'categories'
]

data_precio =[
              'highPrice',
              'lowPrice'
]

#hacer el recorrido por todas las paginas del mercado.
pagina = 2
max_pagina=200
exito = pd.DataFrame()
print('Obteniendo data...')

for pag in paginas:
    print(pag)

    for p in range(1, max_pagina+1):
        if p==1:
            url = 'https://www.exito.com/Mercado/' + pag
        else:
            url = 'https://www.exito.com/mercado/' + pag + '?page=' + str(p)
        driver.get(url)

        driver.quit
        print ('Página #' + str(p))


        #Read lxml in BS4
        soup=BeautifulSoup(driver.page_source, 'lxml')

        productscript=soup.find('script',string=re.compile('"Product:sp'))
        if productscript is None:
            break
        #Convert to json to extract keys and values
        data = json.loads(productscript.get_text())

        temp = {}
        fecha = '2021-10-29' #Fecha de extraccion

        for k in data.keys():
            try:
                
                if data[k]['__typename']=='Product':
                        
                    for i_product in dict_param:
                            
                        if i_product =='categories': # Trae las categorias del json ej, mercado, bebidas
                            x=0
                            for e in data[k][i_product]['json']:
                                temp['categoria' + str(x)]= data[k][i_product]['json'][x]
                                x=x+1
                            
                                
                            
                        else:
                            temp[i_product] = data[k][i_product] #Trae los atributos definidos en dict_param. S epueden configurar mas atributos si se desea
                        
                        temp['fecha'] = fecha
                            
                elif data[k]['__typename']=='PriceRange' and 'sellingPrice' in k: #Trae los precios de ventas. Alto y bajo
                    for i_precio in data_precio:
                        temp[i_precio]= data[k][i_precio]
                elif data[k]['__typename']=='SKU':
                    temp['ean'] = data[k]['ean']

                elif data[k]['__typename']=='Teaser':
                    temp['proveedor_descuento'] = data[k]['name']

                elif data[k]['__typename']=='TeaserValue':
                    temp['proveedor_descuento_valor']=data[k]['value']


                    exito = exito.append(temp, ignore_index=True) #Almacena en un dataframe
                    exito['categoria'] = pag

            except KeyError:
                break

    #Ajusta nombres categorias
    
    # exito['categoria0']=exito.categoria0.str.split('/').str[1]
    # exito['categoria1']=exito.categoria1.str.split('/').str[2]
    # exito['categoria2']=exito.categoria2.str.split('/').str[3]
    exito['almacen']="EXITO"
    exito.proveedor_descuento_valor=exito.proveedor_descuento_valor.astype(float)
    exito.lowPrice = exito.lowPrice * (100 - exito.proveedor_descuento_valor)/100

#Exporta a un csv
nombrearchivo = fecha + ' exito.csv'
exito.to_csv("/content/drive/MyDrive/Vicente/Python/Mercado/" + nombrearchivo, index=False)

In [ ]:
exito

,almacen,brand,categoria0,categoria1,categoria2,ean,fecha,highPrice,link,lowPrice,productId,productName,proveedor_descuento,proveedor_descuento_valor,categoria
0,EXITO,GOURMET,/Mercado/Despensa/Aceites Y Vinagres/,/Mercado/Despensa/,/Mercado/,7702116000013,2021-10-29,18200.0,https://www.exito.com/aceite-girasol-gourmet-f...,17472.0,220465,ACEITE FAMILIA GOURMET 1000 ml,mastercard_25_aliado_mercado_oct29,4.0,snacks
1,EXITO,GOURMET,/Mercado/Despensa/Aceites Y Vinagres/,/Mercado/Despensa/,/Mercado/,7702116000013,2021-10-29,18200.0,https://www.exito.com/aceite-girasol-gourmet-f...,13650.0,220465,ACEITE FAMILIA GOURMET 1000 ml,mastercard_25_aliado_mercado_oct29,25.0,snacks
2,EXITO,HACIENDA GUZMAN MARCA EXCLUSIV,/Mercado/Despensa/Aceites Y Vinagres/,/Mercado/Despensa/,/Mercado/,8410660027767,2021-10-29,61900.0,https://www.exito.com/aceite-de-oliva-extra-vi...,59424.0,73608,Aceite oliva extravirgen HACIENDA GUZMAN MARCA...,mastercard_25_aliado_mercado_oct29,4.0,snacks
3,EXITO,HACIENDA GUZMAN MARCA EXCLUSIV,/Mercado/Despensa/Aceites Y Vinagres/,/Mercado/Despensa/,/Mercado/,8410660027767,2021-10-29,61900.0,https://www.exito.com/aceite-de-oliva-extra-vi...,46425.0,73608,Aceite oliva extravirgen HACIENDA GUZMAN MARCA...,mastercard_25_aliado_mercado_oct29,25.0,snacks
4,EXITO,DIANA,/Mercado/Despensa/Aceites Y Vinagres/,/Mercado/Despensa/,/Mercado/,7702028000378,2021-10-29,30050.0,https://www.exito.com/aceite-diana-vitaminas-x...,28848.0,914228,Aceite Vitaminas DIANA 3000 ml,mastercard_25_aliado_mercado_oct29,4.0,snacks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10423,EXITO,TRULULU,/Mercado/Snacks/Confiteria Y Dulces/,/Mercado/Snacks/,/Mercado/,7702993032206,2021-10-29,6650.0,https://www.exito.com/surtido-barrilete-lokino...,4987.5,840377,SURTIDO BARRILETE + LOKIÑO TRULULU 380 gr,mastercard_25_aliado_mercado_oct29,25.0,snacks
10424,EXITO,SNICKERS,/Mercado/Snacks/Chocolates/,/Mercado/Snacks/,/Mercado/,0040000394129,2021-10-29,5240.0,https://www.exito.com/chocolatina-peanut-butte...,5030.4,759590,Barra de chocolate peanut SNICKERS 50.5 gr,mastercard_25_aliado_mercado_oct29,4.0,snacks
10425,EXITO,SNICKERS,/Mercado/Snacks/Chocolates/,/Mercado/Snacks/,/Mercado/,0040000394129,2021-10-29,5240.0,https://www.exito.com/chocolatina-peanut-butte...,3930.0,759590,Barra de chocolate peanut SNICKERS 50.5 gr,mastercard_25_aliado_mercado_oct29,25.0,snacks
10426,EXITO,MANITOBA,/Mercado/Snacks/Pasabocas De Maiz/,/Mercado/Snacks/,/Mercado/,7702409012655,2021-10-29,50200.0,https://www.exito.com/mix-original-display-591...,48192.0,591045,MIX ORIGINAL DISPLAY MANITOBA 1200 gr,mastercard_25_aliado_mercado_oct29,4.0,snacks


In [ ]:
exito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10428 entries, 0 to 10427
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   almacen                    10428 non-null  object 
 1   brand                      10428 non-null  object 
 2   categoria0                 10428 non-null  object 
 3   categoria1                 10428 non-null  object 
 4   categoria2                 10428 non-null  object 
 5   ean                        10428 non-null  object 
 6   fecha                      10428 non-null  object 
 7   highPrice                  10428 non-null  float64
 8   link                       10428 non-null  object 
 9   lowPrice                   10428 non-null  float64
 10  productId                  10428 non-null  object 
 11  productName                10428 non-null  object 
 12  proveedor_descuento        10428 non-null  object 
 13  proveedor_descuento_valor  10428 non-null  flo